# S&P Stock Puller
The end result will have a queriable local database of all stock history for use with your favorite BI tool.  You will need to sign up on Quandl to get a download key. 
This program will:
    1) Scrape the names of the S&P 500 symbols from Wikipedia
    2) Use Quandl to download the historical stock data
    3) Create a SQLite database table on your machine, and load the stock data into it.

In [49]:
import urllib2 
import urllib
from bs4 import BeautifulSoup
import sqlite3
import json

QuandlKey =     #put your Quandl key in as a string: 'quandlkey' 

### Scrape list of stocks from wikipedia
#### code lifted from http://www.thealgoengineer.com/2014/download_sp500_data/
Creates a dictionary of sector types, e.g. 'consumer_discretionary', 'energy' etc.  Each sector is referenced to a list of ticker symbols within that sector.  

In [9]:
def scrape_list(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(site, headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page, 'lxml')

    table = soup.find('table', {'class': 'wikitable sortable'})
    sector_tickers = dict()
    for row in table.findAll('tr'):
        col = row.findAll('td')
        if len(col) > 0:
            sector = str(col[3].string.strip()).lower().replace(' ', '_')
            ticker = str(col[0].string.strip())
            if sector not in sector_tickers:
                sector_tickers[sector] = list()
            sector_tickers[sector].append(ticker)
    return sector_tickers

In [10]:
SITE = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sector_tickers = scrape_list(SITE)  

### Create a connection to a SQLite database on your machine
Will make a SQLite database, which can be used as an ODBC connection. Creates a OHLC table.  


In [59]:
#Fire up db connection
db='Test_Stock.db'  #Running first time will create this file in your directory
datatable = 'OHLC' #name of table to be created and written to
conn = sqlite3.connect(db)
c = conn.cursor()

In [60]:
##Create Table within the database
c.execute('CREATE TABLE IF NOT EXISTS {tn} (Symbol TEXT, Sector Text, Date TEXT,  \
           Open REAL, High REAL, Low REAL, Close REAL, Volume REAL, \
          Ex_Dividend REAL, Split_Ratio REAL, Adj_Open REAL, Adj_High REAL,\
          Adj_Low REAL, Adj_Close REAL, Adj_Volume REAL)'.format(tn=datatable))

### Create few functions to query quandl and write to database 

In [57]:
#deterimines if your database already has data from a symbol
def does_db_have_symbol(symbol,datatable):  
    c.execute("SELECT Count(*) FROM {tn} WHERE Symbol ='{sn}' ".format(tn=datatable, sn=symbol))
    x= c.fetchone()[0]
    out= True
    if x==0:
        print '%s does not exist in database' %symbol
        out = False
    return out
    
# queriesl Quandl for entire symbol history
# Has issues if wiki sybmol doesn't match Quandl, e.g.,  BF_B, BRK_B and UA.C
def get_whole_symbol_table(symbol):
    if symbol=='BF-B':
        symbol='BF_B'
    if symbol=='BRK-B':
        symbol='BRK_B'
    if symbol != 'UA.C':
        jsontext= urllib.urlopen('https://www.quandl.com/api/v3/datasets/WIKI/' +symbol+ '.json?api_key='+QuandlKey)
        print 'qeurying Quandl for %s' %symbol
        data=json.load(jsontext)
        print 'returning %s data' %symbol
        return data
    else:
        print'skipping UA.C'

#writes the quandl query rows into the created database table
def write_daily_rows(data, symbol, sector):
    if symbol != 'UA.C':
        print 'writing to database'
        for row in data['dataset']['data']: 
            c.execute("INSERT INTO OHLC VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",(symbol,sector,
                                                                            row[0],row[1], row[2], row[3], 
                                                                            row[4], row[5], row[6],row[7], row[8],row[9],row[10],row[11],row[12]))



### Populate your database table with stock data
Takes a while to complete ~1 hour
Program will:
1) cycle through all the ticker symbols
2) download each symbols data from Quandl
3) write the stock data to your data table 

In [62]:
for sector in sector_tickers.keys():
    for stock in sector_tickers[sector]:
        print stock, sector
        if does_db_have_symbol(stock,datatable) is False:
            data=get_whole_symbol_table(stock)
            write_daily_rows(data, stock,sector)
            conn.commit()

MMM industrials
MMM does not exist in database
qeurying Quandl for MMM
returning MMM data
writing to database
AYI industrials
AYI does not exist in database
qeurying Quandl for AYI
returning AYI data
writing to database
ALK industrials
ALK does not exist in database
qeurying Quandl for ALK
returning ALK data
writing to database
ALLE industrials
ALLE does not exist in database
qeurying Quandl for ALLE
returning ALLE data
writing to database
AAL industrials
AAL does not exist in database
qeurying Quandl for AAL
returning AAL data
writing to database
AME industrials
AME does not exist in database
qeurying Quandl for AME
returning AME data
writing to database
APH industrials
APH does not exist in database
qeurying Quandl for APH
returning APH data
writing to database
ARNC industrials
ARNC does not exist in database
qeurying Quandl for ARNC
returning ARNC data
writing to database
BA industrials
BA does not exist in database
qeurying Quandl for BA
returning BA data
writing to database
CHRW i